# Quickstart Guide: 

This Quickstart Guide presents a simple example of **ocean data challenge** for mapping the Sea Surface Height from sparse observations. 

The methodology is based on an Observing System Simulation Experiment (OSSE). The inputs data represent altimeter observations extracted from a realistic high-resolution ocean model simulation (NATL60). A simple mapping algorithm (Optimal Interpolation) is used to produce the reconstructed SSH field from the sparse observations. Finally, a comparison between the reconstructed and the reference SSH fields is done to quantify the reconstruction scores.

Three experiments are carried out with <font color=grey> **Experiment 3**</font>: demo. of reconstruction with <font color=grey> **1 SWOT altimeter**</font>

The notebook is structured as follows:

    1) downloading the data
    2) Setup configuration of the interpolation
    3) Run the DINAE experiment
    4) Plot some figures and interpolation scores

This quickstart guide take approx. 30 min to run on a PC. 

In [1]:
import sys
sys.path.append("../")
from DINAE import *

Initializing DINAE libraries...


/home/administrateur/anaconda3/envs/conda-forge/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


...Done


Using TensorFlow backend.


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
import dask
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:38729 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.63 GB


### 1- DOWNLOADING DATA

When downloading the 3 datasets (ref, data & oi), be sure that the names of the files will be the same than those specified in the config.yml file used to define the setup configuration. If no, please modify the config.yml file or change the downloaded file names.

##### Download Nature run SSH for mapping evaluation

In [ ]:
!wget https://s3.eu-central-1.wasabisys.com/melody/REF.nc -O "ref.nc"

##### Download Synthetic SSH observation for OI mapping

In [ ]:
!wget https://s3.eu-central-1.wasabisys.com/melody/DATA.nc -O "data.nc"

#####  Download OI mapping

In [ ]:
!wget https://s3.eu-central-1.wasabisys.com/melody/OI.nc -O "oi.nc"

### 2- SETUP CONFIGURATION 

In [4]:
# description of the parameters
domain   = "GULFSTREAM"
opt      = "nadirswot"
lag      = "0"             
type_obs = "mod"       # mod/obs

# main code
parser = argparse.ArgumentParser()
with open('config.yml', 'rb') as f:
    conf = yaml.load(f.read())
# list of global parameters (comments to add)
fileMod                 = conf['path_files']['fileMod']
fileOI                  = conf['path_files']['fileOI']
fileObs                 = conf['path_files']['fileObs']
flagTrWMissingData      = conf['data_options']['flagTrWMissingData'] 
flagloadOIData          = conf['data_options']['flagloadOIData']
include_covariates      = conf['data_options']['include_covariates']
lfile_cov               = conf['data_options']['lfile_cov']
lname_cov               = conf['data_options']['lname_cov']
lid_cov                 = conf['data_options']['lid_cov']   
N_cov                   = ifelse(include_covariates==True,len(lid_cov),0)
size_tw                 = conf['data_options']['size_tw'] 
Wsquare                 = conf['data_options']['Wsquare']
Nsquare                 = conf['data_options']['Nsquare']
DimAE                   = conf['NN_options']['DimAE']
flagAEType              = conf['NN_options']['flagAEType']
sigNoise                = conf['data_options']['sigNoise']
flagTrOuputWOMissingData= conf['data_options']['flagTrOuputWOMissingData']
stdMask                 = conf['data_options']['stdMask']
dropout                 = conf['data_options']['dropout']
wl2                     = conf['data_options']['wl2']
batch_size              = conf['training_params']['batch_size']
NbEpoc                  = conf['training_params']['NbEpoc']
Niter                   = conf['training_params']['Niter']

### 3- CREATE OUTPUT DIRECTORY

In [5]:
if flagAEType==1:
    suf1 = "ConvAE"
if flagAEType==2:
    suf1 = "GENN"
if flagTrWMissingData==0:
    suf2 = "womissing"
elif flagTrWMissingData==1:
    suf2 = "wmissing"
else:
    suf2 = "wwmissing"
suf3 = "FP"
suf4 = ifelse(include_covariates==True,"w"+'-'.join(lid_cov),"wocov")
dirSAVE = '.'

### 4- RUN EXPERIMENTS

#### 4.1 Read the data

In [6]:
# push all global parameters in a list
def createGlobParams(params):
    return dict(((k, eval(k)) for k in params))
list_globParams=['domain','fileMod','fileObs','fileOI',\
'include_covariates','N_cov','lfile_cov','lid_cov','lname_cov',\
'flagTrOuputWOMissingData','flagTrWMissingData',\
'flagloadOIData','size_tw','Wsquare',\
'Nsquare','DimAE','flagAEType',\
'sigNoise','stdMask','dropout','wl2','batch_size',\
'NbEpoc','Niter','dirSAVE','suf1','suf2','suf3','suf4']
globParams = createGlobParams(list_globParams)   

## 1) *** Read the data ***
genFilename, x_train, y_train, mask_train, gt_train, x_train_missing, meanTr, stdTr,\
x_test, y_test, mask_test, gt_test, x_test_missing, lday_test, x_train_OI, x_test_OI = import_Data(globParams,type_obs)

1) .... Load SSH dataset (training data): /data/dataset_nadir_0d_swot.nc


FileNotFoundError: [Errno 2] No such file or directory: b'/ref/NATL60-CJM165_GULFSTREAM_ssh_y2013.1y.nc'

#### 4.2 Define the NN architecture

In [ ]:
genFilename, encoder, decoder, model_AE, DIMCAE = define_Models(globParams,genFilename,x_train,mask_train)

#### 4.3 Train DINAE with Fixed-Point solver    

In [ ]:
FP_solver(globParams,genFilename,x_train,x_train_missing,mask_train,gt_train,meanTr,stdTr,\
          x_test,x_test_missing,mask_test,gt_test,lday_test,x_train_OI,x_test_OI,encoder,decoder,model_AE,DIMCAE)

### 5 - POSTPROCESSING

#### 5.1 Plot figures    

#### 5.2 Compute RMSE and spectrum   